# ExecTimer

In [10]:
import time

import sys
sys.path.append('..')

from astroLuSt.monitoring import timers

import importlib
importlib.reload(timers)

<module 'astroLuSt.monitoring.timers' from 'c:\\Users\\lukas\\Documents\\GitHub\\astroLuSt\\demos\\..\\astroLuSt\\monitoring\\timers.py'>

In [11]:
ET = timers.ExecTimer(verbose=1)

ET.checkpoint_start('Sup Task', comment='Global Task')

ET.checkpoint_start('Sub Task1')
time.sleep(2)
ET.checkpoint_end('Sub Task1')
ET.checkpoint_start('Sub Task2')
time.sleep(3)
ET.checkpoint_end('Sub Task2')

ET.checkpoint_end('Sup Task')

print(ET.df_protocoll)



######################################################################
INFO: Started Sup Task at 2023-03-04T14:49:57

######################################################################
INFO: Started Sub Task1 at 2023-03-04T14:49:57

INFO: Finished Sub Task1 at 2023-03-04T14:49:59
Required time: 0 days 00:00:02
######################################################################

######################################################################
INFO: Started Sub Task2 at 2023-03-04T14:49:59

INFO: Finished Sub Task2 at 2023-03-04T14:50:02
Required time: 0 days 00:00:03
######################################################################

INFO: Finished Sup Task at 2023-03-04T14:50:02
Required time: 0 days 00:00:05
######################################################################
        Task               Start                  End         Duration  \
0   Sup Task 2023-03-04 14:49:57  2023-03-04T14:50:02  0 days 00:00:05   
1  Sub Task1 2023-03-04 14:49:57  2023-03-04